In [53]:
from tabula import read_pdf
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt

In [2]:
def get_emissions_table_from_page_1353():
    """Page 1353 from the PDF document, is 1335 IN the document!"""
    df = read_pdf("/home/martin/Downloads/ipcc_wg3_ar5_full.pdf", pages=1353)[0]
    return df

In [3]:
df = get_emissions_table_from_page_1353()

In [14]:
df

,Options,Direct emissions,Infrastructure & supply\rchain emissions,Biogenic CO2 emissions\rand albedo effect,Methane emissions,Lifecycle emissions\r(incl. albedo effect)
0,NaN,Min / Median / Max,Typical values,Min / Median / Max,NaN,NaN
1,Currently Commercially Available Technologies,NaN,NaN,NaN,NaN,NaN
2,Coal — PC,670 / 760 / 870,9.6,0,47,740 / 820 / 910
3,Gas — Combined Cycle,350 / 370 / 490,1.6,0,91,410 / 490 / 650
4,Biomass — cofiring,n. a. ii,–,–,–,620 / 740 / 890iii
5,Biomass — dedicated,n. a. ii,210,27,0,130 / 230 / 420iv
6,Geothermal,0,45,0,0,6.0 / 38 / 79
7,Hydropower,0,19,0,88,1.0 / 24 / 2200
8,Nuclear,0,18,0,0,3.7 / 12 / 110
9,Concentrated Solar Power,0,29,0,0,8.8 / 27 / 63


In [135]:
def get_lifecycle_emissions_df(df):
    df_split_lifecycle = df["Lifecycle emissions\r(incl. albedo effect)"].str.extract('(\d+) / (\d+) / (\d+)')
    df_split_lifecycle = df_split_lifecycle.columns = ["Min", "Median", "Max"]
    df_split_lifecycle = df_split_lifecycle.apply(pd.to_numeric, errors="coerce")
    lifecycle_df = pd.concat([df["Options"], df_split_lifecycle], axis=1).dropna(how="all")
    return lifecycle_df
    

In [132]:
lifecycle_df = get_lifecycle_emissions_df(df)

In [133]:
lifecycle_df

,Options,Min,Median,Max
1,Currently Commercially Available Technologies,NaN,NaN,NaN
2,Coal — PC,740,820,910
3,Gas — Combined Cycle,410,490,650
4,Biomass — cofiring,620,740,890
5,Biomass — dedicated,130,230,420
6,Geothermal,0,38,79
7,Hydropower,0,24,2200
8,Nuclear,7,12,110
9,Concentrated Solar Power,8,27,63
10,Solar PV — rooftop,26,41,60


In [111]:
lifecycle_df[cols].apply(pd.to_numeric, errors="coerce").

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [13]:
# Visualize DataFrame

In [101]:
dropna_lc_df = lifecycle_df.dropna()

In [102]:
coal_df = dropna_lc_df.iloc[0:2]

In [103]:
coal_df.columns

Index(['Options', 'Min', 'Median', 'Max'], dtype='object')

In [104]:
sns.set(style='whitegrid')
plt.figure(figsize=(8, 6))

# Plot the bars for min, median, and max
sns.barplot(x='Options', y='Min', data=coal_df, color='lightblue', label='Min')
sns.barplot(x='Options', y='Median', data=coal_df, color='white', edgecolor='black', hatch='--', label='Median')
sns.barplot(x='Options', y='Max', data=coal_df, color='lightblue', label='Max')

# Set labels and title
plt.xlabel('Options')
plt.ylabel('Values')
plt.title('Min, Median, and Max Values for Coal and Gas')

# Add legend
plt.legend()

# Show the plot
plt.show()

TypeError: Neither the `x` nor `y` variable appears to be numeric.

<Figure size 800x600 with 0 Axes>

In [71]:
coal_df.dtypes

Options    object
Min        object
Median     object
Max        object
dtype: object

In [58]:
# Set up the plot
sns.set(style='whitegrid')
plt.figure(figsize=(10, 6))

# Plot the bars for min, median, and max
sns.barplot(x='Options', y='Min', data=dropna_lc_df, color='lightblue', label='Min')
sns.barplot(x='Options', y='Median', data=dropna_lc_df, color='white', edgecolor='black', hatch='--', label='Median')
sns.barplot(x='Options', y='Max', data=dropna_lc_df, color='lightblue', label='Max')
#
# Set labels and title
plt.xlabel('Options')
plt.ylabel('Emissions (gCO2/kWh)')
plt.title('Ranges of Emissions')

# Add legend
plt.legend()

# Show the plot
plt.show()

TypeError: Neither the `x` nor `y` variable appears to be numeric.

<Figure size 1000x600 with 0 Axes>